# **Part 3: Developing an Agentic AI Travel Assistant**

In [ ]:
pip install crewai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.5/285.5 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.4 MB/s eta 0:00

### **Create tools required for extracting data from external APIs**

Create the follwoing tools based on BaseTool class from crewai.tools library to fetch real time data required for the AI Agents:

**AmadeusFlightApiTool:**
Fetches real time data for flights availble between the departure date and return date for a given origin city and destination city using the external API provided by Amadeus.com

**BookingApiTool**:
Fetches real time data for hotels available for reservation between departure date and return date at a given destination city using the external API provided by Booking.com

**OpenWeatherApiTool**:
Featches real time data for weather forecast between the departure date and return date for a given destination city using the external API provided by Openweathermap.org

In [ ]:
from crewai.tools import BaseTool
from datetime import datetime
from openai import OpenAI
from pydantic import BaseModel
from typing import Type
import requests

class AmadeusFlightApiTool(BaseTool):
    name:str = "AmadeusFlightAPI"
    description:str = "Fetches flight offers from Amadeus"

    def get_token(self, client_id, client_secret):
        url = "https://test.api.amadeus.com/v1/security/oauth2/token"
        response = requests.post(url, data={
            "grant_type": "client_credentials",
            "client_id": client_id,
            "client_secret": client_secret
        })

        return response.json()["access_token"]

    def extract_flight_info(self, response):
        flights = []

        for offer in response.get('data', []):
            flight_id = offer['id']
            total_price = offer['price']['total']
            currency = offer['price']['currency']
            airline = offer['validatingAirlineCodes'][0]
            seat_count = offer['numberOfBookableSeats']
            round_trip = not offer['oneWay']

            itinerary_data = []
            for itinerary in offer['itineraries']:
                segments_info = []
                for segment in itinerary['segments']:
                    segments_info.append({
                        'from': segment['departure']['iataCode'],
                        'to': segment['arrival']['iataCode'],
                        'departure_time': segment['departure']['at'],
                        'arrival_time': segment['arrival']['at'],
                        'duration': segment['duration'],
                        'flight_number': f"{segment['carrierCode']} {segment['number']}",
                        'aircraft': segment['aircraft']['code'],
                        'stops': segment['numberOfStops']
                    })
                itinerary_data.append({
                    'duration': itinerary['duration'],
                    'segments': segments_info
                })

            flights.append({
                'flight_id': flight_id,
                'total_price': f"{currency} {total_price}",
                'airline': airline,
                'seat_count': seat_count,
                'round_trip': round_trip,
                'itineraries': itinerary_data,
            })

        return flights

    def _run(self, origin: str, destination: str, departure_date: str, return_date: str) -> str:
        client_id = AMADEUS_CLIENT_ID
        client_secret = AMADEUS_CLIENT_SECRET
        token = self.get_token(client_id, client_secret)

        url = "https://test.api.amadeus.com/v2/shopping/flight-offers"
        headers = {"Authorization": f"Bearer {token}"}
        params = {
            "originLocationCode": origin,
            "destinationLocationCode": destination,
            "departureDate": departure_date,
            "returnDate": return_date,
            "adults": 1,
            "max": 2,
            "nonStop": 'true',
            "currencyCode": "USD"
        }
        response = requests.get(url, headers=headers, params=params)
        data = response.json()

        return self.extract_flight_info(data)

class BookingApiTool(BaseTool):
    name:str  = "BookingAPI"
    description:str  = "Finds hotels using Booking.com via RapidAPI"

    def get_destination_id(self, destination):
        rapidapi_key = RAPIDAPI_KEY
        url = "https://booking-com15.p.rapidapi.com/api/v1/hotels/searchDestination?"
        headers = {
            "X-RapidAPI-Key": rapidapi_key,
            "X-RapidAPI-Host": "booking-com15.p.rapidapi.com"
        }
        params = {
            "query": destination
        }
        response = requests.get(url, headers=headers, params=params)
        data = response.json()
        for item in data.get("data", []):
            if item.get("search_type") == "city":
                return item["dest_id"]


    def _run(self, destination: str, departure_date: str, return_date: str) -> str:
        rapidapi_key = RAPIDAPI_KEY
        dest_id = self.get_destination_id(destination)


        url = "https://booking-com15.p.rapidapi.com/api/v1/hotels/searchHotels"

        headers = {
            "X-RapidAPI-Key": rapidapi_key,
            "X-RapidAPI-Host": "booking-com15.p.rapidapi.com"
        }
        params = {
            "arrival_date": departure_date,
            "departure_date": return_date,
            "units": "metric",
            "search_type": "city",
            "order_by": "popularity",
            "adults_number": 1,
            "dest_id": dest_id,
            "locale": "en-us",
            "currency": "USD"
        }
        response = requests.get(url, headers=headers, params=params)
        data = response.json().get("data")
        hotels = data.get("hotels", [])
        results = []
        for hotel in hotels[:3]:
            name = hotel.get("accessibilityLabel")
            results.append(f"{name}")
        return "\n\n".join(results)

class OpenWeatherApiTool(BaseTool):
    name:str  = "OpenWeatherAPI"
    description:str  = "Fetches weather forecast using OpenWeatherMap"

    def _run(self, destination: str, departure_date: str, return_date: str) -> str:
        api_key = OPENWEATHER_API_KEY
        url = "https://api.openweathermap.org/data/2.5/forecast"
        params = {
            "q": destination,
            "appid": api_key,
            "units": "metric"
        }
        res = requests.get(url, params=params)
        res.raise_for_status()
        data = res.json()

        forecasts = []
        for entry in data.get("list", []):
            forecast_time = datetime.fromtimestamp(entry["dt"])
            date_str = forecast_time.strftime("%Y-%m-%d")

            if departure_date <= date_str <= return_date:
                temp = entry["main"]["temp"]
                desc = entry["weather"][0]["description"]
                forecasts.append(f"{date_str} {forecast_time.strftime('%H:%M')} - {temp}°C, {desc}")

        return "\n".join(forecasts) if forecasts else "No forecast data available for the given range."

### **Create Agents and Tools**

Create the following agents and define associated tasks:

**Flight Agent:**
A flight information specialist and finds a list of best suited flights for the travel dates with details such as pricing, capacity, duration, airline and airport information using Amadeus Flight API Tool.

- **Task:**
Retrieve available flights based on the origin/destination city and travel dates.

**Hotel Agent:**
A travel booking expert and gathers best hotel accomodations availble for the travel dates in the destination city and provides information such as pricing, availability, user ratings, distance from city centre, and hotel policies using Booking API Tool.

- **Task**
Find accommodation options at the destination city for given travel dates.

**Weather Agent**
A weather forecast specialist and provides weather forecast information for the travel dates at the destination city.

- **Task**
Fetch the weather forecast for the travel destination on the specified dates

**Travel Itenray Planner Agent**
A seasoned travel planner and creates a detailed travel itenary to visit a distination city for the given travel dates by interacting with other agent, collecting all the necessary information, and generates an day to day itenary including details suchs as flight details, weather forecast, hotel details, summarizes key information and provides recommendations such as nearby palces of attactions and activities.

- **Task**
Synthesize information from the other agents to generate a personalized
travel itinerary.





In [ ]:
from crewai import Agent, Task

flight_api_agent = Agent(
    role="Flight Information Specialist",
    goal="Find the best available flights based on destination, departure_date, return_date, origin.",
    backstory="An expert in aviation logistics and airfare pricing, using data fetched from Amadeus API.",
    verbose=True,
    tools=[AmadeusFlightApiTool()]
)

hotel_api_agent = Agent(
    role="Hotel Booking Assistant",
    goal="Find and suggest hotels based on based on destination, departure_date, and return_date.",
    backstory="A travel booking expert who uses Booking.com or Expedia to find top-rated accommodations.",
    tools=[BookingApiTool()],
    verbose=True
)

weather_api_agent = Agent(
    role="Weather Forecast Specialist",
    goal="Provide an accurate weather forecast for the travel destination, departure_date, and return_date.",
    backstory="A weather enthusiast who uses OpenWeatherMap to deliver detailed weather updates.",
    tools=[OpenWeatherApiTool()],
    verbose=True
)

itinerary_planner_agent = Agent(
    role="Travel Itinerary Planner",
    goal="Combine all travel information into a personalized travel itinerary using outputs from Flight, Hotel and Weather agents.",
    backstory="A seasoned travel planner who curates trips by combining flight, hotel, and weather data and provides a recommended day by day trip itenary",
    tools=[],
    verbose=True
)

flight_task = Task(
    description="Retrieve available flights from the Amadeus API for the user's specified Origin: {origin}, Destination: {destination}, Departure Date: {departure_date}, Return Date: {return_date}. Include airline, departure/arrival times, and pricing.",
    expected_output="A list of available flights with price, origin and destination airport codes, timings, and costs.",
    agent=flight_api_agent
)

hotel_task = Task(
    description="Use the Booking API to search for hotels in the {destination}, for the {departure_date} and {return_date}. Return hotel names, availability, prices, and user ratings.",
    expected_output="List of hotels with pricing, availability, and ratings.",
    agent=hotel_api_agent
)

weather_task = Task(
    description="Fetch the weather forecast for the {destination} on the specified {departure_date}, {return_date} using data fetched fromm OpenWeatherMap API. Include temperature, condition (e.g., sunny, rainy), and a daily summary.",
    expected_output="Daily weather summary with temperature and condition.",
    agent=weather_api_agent
)

itinerary_task = Task(
    description="Creates a travel itinerary using results from the Flight, Weather, and Hotel agents. Summarize all key information clearly and provide a recommended day by day travel itenary.",
    expected_output="A complete travel itinerary including flight details, weather forecast, and hotel details and provide a recommended day by day travel itenary.",
    agent=itinerary_planner_agent,
    context=[flight_task, hotel_task, weather_task]
)

### **Creating a crew of Agents**

Using the crew ai library create a crew of agents consisting of Flight Agent, Hotel Agent, Weather Agent, and Iternary Planner Agent. Assign the corresponing tasks for the agents

In [ ]:
from crewai import Crew, Process

travel_planning_crew = Crew(
    agents=[flight_api_agent, hotel_api_agent, weather_api_agent, itinerary_planner_agent],
    tasks=[flight_task, hotel_task, weather_task, itinerary_task],
    process=Process.sequential,
    verbose=True
)

### **Generate the Travel Itenray for sample inputs using the crew of agents**

### Query 1:

In [ ]:
origin = input("Enter origin city: ")
destination = input("Enter destination city: ")
departure_date = input("Enter departure date (YYYY-MM-DD): ")
return_date = input("Enter return date (YYYY-MM-DD): ")

inputs = {
    "origin": origin,
    "destination": destination,
    "departure_date": departure_date,
    "return_date": return_date
}

Enter origin city: San Francisco
Enter destination city: Los Angeles
Enter departure date (YYYY-MM-DD): 2025-04-25
Enter return date (YYYY-MM-DD): 2025-04-27


In [ ]:
results = travel_planning_crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 4df25ac7-3665-4a1d-9ee4-14c81d7ec24c                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 4cd36120-8f46-402b-b55e-8188f2960307
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 4cd36120-8f46-402b-b55e-8188f2960307
       Status: Executing Task...
    └── 🤖 Agent: Flight Information Specialist
            Status: In Progress

# Agent: Flight Information Specialist
## Task: Retrieve available flights from the Amadeus API for the user's specified Origin: San Francisco, Destination: Los Angeles, Departure Date: 2025-04-25, Return Date: 2025-04-27. Include airline, departure/arrival times, and pricing.


🤖 Agent: Flight Information Specialist
    Status: In Progress

...

# Agent: Travel Itinerary Planner
## Task: Creates a travel itinerary using results from the Flight, Weather, and Hotel agents. Summarize all key information clearly and provide a recommended day by day travel itenary.


# Agent: Travel Itinerary Planner
## Final Answer: 
### Travel Itinerary: San Francisco to Los Angeles

#### Overview:
- **Destination:** Los Angeles, California
- **Trip Duration:** April 25, 2025 - April 27, 2025
- **Flight Options:** 
  - **Flight ID 1:** San Francisco (SFO) to Los Angeles (LAX)
  - **Flight ID 2:** San Francisco (SFO) to Ontario (ONT)
- **Hotel Options:** 
  - Crowne Plaza Hotel Los Angeles Harbor 
  - LA Crystal Hotel 
  - The Anza - a Calabasas Hotel
- **Weather Forecast:** Cool and cloudy with slight rain.

---

### Flight Details:

#### Departure Flight:
- **Airline:** F9
- **Departure:** April 25, 2025, at 14:20 
- **From:** SFO (San Francisco)
- **Arrival:** April 25, 2025, at 15:59 
- **To:** LAX (Los Angeles)
- **Duration:** 1 hour 39 minut

🚀 Crew: crew
├── 📋 Task: 4cd36120-8f46-402b-b55e-8188f2960307
│      Assigned to: Flight Information Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Flight Information Specialist
│           Status: ✅ Completed
├── 📋 Task: 95443b32-1711-420d-93ba-f3422773a183
│      Assigned to: Hotel Booking Assistant
│      Status: ✅ Completed
│   └── 🤖 Agent: Hotel Booking Assistant
│           Status: ✅ Completed
├── 📋 Task: cd8e8d7d-9d5c-44bf-8d30-5a45f4583038
│      Assigned to: Weather Forecast Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Weather Forecast Specialist
│           Status: ✅ Completed
└── 📋 Task: 35381d32-abb8-4dfb-9e65-08ba9822929f
       Status: Executing Task...
    └── 🤖 Agent: Travel Itinerary Planner
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 4cd36120-8f46-402b-b55e-8188f2960307
│      Assigned to: Flight Information Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Flight Information Specialist
│           Status: ✅ Completed
├── 📋 Task: 95443b32-1711-420d-93ba-f3422773a183
│      Assigned to: Hotel Booking Assistant
│      Status: ✅ Completed
│   └── 🤖 Agent: Hotel Booking Assistant
│           Status: ✅ Completed
├── 📋 Task: cd8e8d7d-9d5c-44bf-8d30-5a45f4583038
│      Assigned to: Weather Forecast Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Weather Forecast Specialist
│           Status: ✅ Completed
└── 📋 Task: 35381d32-abb8-4dfb-9e65-08ba9822929f
       Assigned to: Travel Itinerary Planner
       Status: ✅ Completed
    └── 🤖 Agent: Travel Itinerary Planner
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 35381d32-abb8-4dfb-9e65-08ba9822929f                                                                     │
│  Agent: Travel Itinerary Planner                                                                                │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 4df25ac7-3665-4a1d-9ee4-14c81d7ec24c                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
from IPython.display import Markdown, display

result = results.tasks_output[-1]
display(Markdown(result.raw))

### Travel Itinerary: San Francisco to Los Angeles

#### Overview:
- **Destination:** Los Angeles, California
- **Trip Duration:** April 25, 2025 - April 27, 2025
- **Flight Options:** 
  - **Flight ID 1:** San Francisco (SFO) to Los Angeles (LAX)
  - **Flight ID 2:** San Francisco (SFO) to Ontario (ONT)
- **Hotel Options:** 
  - Crowne Plaza Hotel Los Angeles Harbor 
  - LA Crystal Hotel 
  - The Anza - a Calabasas Hotel
- **Weather Forecast:** Cool and cloudy with slight rain.

---

### Flight Details:

#### Departure Flight:
- **Airline:** F9
- **Departure:** April 25, 2025, at 14:20 
- **From:** SFO (San Francisco)
- **Arrival:** April 25, 2025, at 15:59 
- **To:** LAX (Los Angeles)
- **Duration:** 1 hour 39 minutes
- **Price:** USD 177.98

#### Return Flight:
- **Airline:** F9
- **Departure:** April 27, 2025, at 18:55 
- **From:** LAX (Los Angeles)
- **Arrival:** April 27, 2025, at 20:37 
- **To:** SFO (San Francisco)
- **Duration:** 1 hour 42 minutes

---

### Hotel Accommodation:

#### Recommended Hotel:
- **Crowne Plaza Hotel Los Angeles Harbor by IHG**
  - **Rating:** 4 out of 5 stars
  - **User Rating:** 7.8 (Good) from 1172 reviews
  - **Distance from Center:** 35 km
  - **Room:** 2 beds
  - **Price:** 509 USD (includes taxes and charges)
  - **Cancellation Policy:** Free cancellation (no prepayment needed)

*Alternative Hotel Options:*
- **LA Crystal Hotel** 
  - **Rating:** 3 out of 5 stars / User Rating: 8.0 (Very good)
  - **Current Price:** 323 USD 

---

### Weather Forecast:

#### April 25, 2025:
- **Temperature:** Ranges from 12.96°C to 17.56°C with a peak at 15:00 at 13.03°C.
- **Conditions:** Broken clouds and overcast throughout the day.
- **Summary:** A cool day with overcast skies.

#### April 27, 2025:
- **Temperature:** Ranges from 11.49°C to 15.81°C.
- **Conditions:** Light rain expected in the early hours, then clearing skies in the afternoon.
- **Summary:** Initial light rain followed by clearer weather.

---

### Recommended Day by Day Itinerary:

#### **Day 1: April 25, 2025:** Arrival in Los Angeles
- **Morning**: Depart from San Francisco (SFO) at 14:20. 
- **Afternoon**: Arrive at Los Angeles (LAX) at 15:59.
- **Check-in**: Crowne Plaza Hotel by 16:30.
- **Evening**: Have a leisurely dinner at a local restaurant, then explore the nearby waterfront.

#### **Day 2: April 26, 2025:** Exploring Los Angeles
- **Morning**: Breakfast at the hotel.
- **Midday**: Visit attractions like the Getty Center or Griffith Observatory. Be prepared for cloudy weather, so dress warmly and carry an umbrella.
- **Afternoon**: Lunch at a local café, followed by a stroll around Santa Monica Pier.
- **Evening**: Return to your hotel to relax; consider a night out in downtown LA for dinner.

#### **Day 3: April 27, 2025:** Departure
- **Morning**: Early breakfast, light rain expected, prepare for weather.
- **Midday**: Last-minute shopping or sightseeing, perhaps visit a nearby park if conditions improve.
- **Afternoon**: Check out of the hotel and make your way to LAX.
- **Departure Flight**: Leave Los Angeles (LAX) at 18:55, arriving in San Francisco (SFO) at 20:37.

---

### Enjoy your trip to Los Angeles!

### Query 2:

In [ ]:
origin = input("Enter origin city: ")
destination = input("Enter destination city: ")
departure_date = input("Enter departure date (YYYY-MM-DD): ")
return_date = input("Enter return date (YYYY-MM-DD): ")

inputs = {
    "origin": origin,
    "destination": destination,
    "departure_date": departure_date,
    "return_date": return_date
}

Enter origin city: Phoenix
Enter destination city: Portland
Enter departure date (YYYY-MM-DD): 2025-04-26
Enter return date (YYYY-MM-DD): 2025-04-29


In [ ]:
results = travel_planning_crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 4df25ac7-3665-4a1d-9ee4-14c81d7ec24c                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 4cd36120-8f46-402b-b55e-8188f2960307
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 4cd36120-8f46-402b-b55e-8188f2960307
       Status: Executing Task...
    └── 🤖 Agent: Flight Information Specialist
            Status: In Progress

# Agent: Flight Information Specialist
## Task: Retrieve available flights from the Amadeus API for the user's specified Origin: Phoenix, Destination: Portland, Departure Date: 2025-04-26, Return Date: 2025-04-29. Include airline, departure/arrival times, and pricing.


🤖 Agent: Flight Information Specialist
    Status: In Progress

...

# Agent: Travel Itinerary Planner
## Task: Creates a travel itinerary using results from the Flight, Weather, and Hotel agents. Summarize all key information clearly and provide a recommended day by day travel itenary.


# Agent: Travel Itinerary Planner
## Final Answer: 
**Travel Itinerary for Portland, Oregon (April 26 - April 29, 2025)**

---

**Flight Details:**

- **Departure:**
   - **Airline:** Hawaiian Airlines (HA)
   - **From:** Phoenix (PHX)
   - **To:** Portland (PDX)
   - **Date:** April 26, 2025
   - **Time:** 07:00 
   - **Arrival Time:** 09:50 
   - **Duration:** 2 hours 50 minutes
   - **Total Price:** USD 496.58
   - **Available Seats:** 7

- **Return:**
   - **Departure:** Portland (PDX)
   - **Arrival:** Phoenix (PHX)
   - **Date:** April 29, 2025
   - **Time:** 17:25 
   - **Arrival Time:** 19:58 
   - **Duration:** 2 hours 33 minutes

---

**Hotel Accommodation:**

- **Staypineapple, Hotel Rose, Downtown Portland**
   - **Rating:** 4 out of 5 stars
   - **User Rat

🚀 Crew: crew
├── 📋 Task: 4cd36120-8f46-402b-b55e-8188f2960307
│      Assigned to: Flight Information Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Flight Information Specialist
│           Status: ✅ Completed
├── 📋 Task: 95443b32-1711-420d-93ba-f3422773a183
│      Assigned to: Hotel Booking Assistant
│      Status: ✅ Completed
│   └── 🤖 Agent: Hotel Booking Assistant
│           Status: ✅ Completed
├── 📋 Task: cd8e8d7d-9d5c-44bf-8d30-5a45f4583038
│      Assigned to: Weather Forecast Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Weather Forecast Specialist
│           Status: ✅ Completed
└── 📋 Task: 35381d32-abb8-4dfb-9e65-08ba9822929f
       Status: Executing Task...
    └── 🤖 Agent: Travel Itinerary Planner
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 4cd36120-8f46-402b-b55e-8188f2960307
│      Assigned to: Flight Information Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Flight Information Specialist
│           Status: ✅ Completed
├── 📋 Task: 95443b32-1711-420d-93ba-f3422773a183
│      Assigned to: Hotel Booking Assistant
│      Status: ✅ Completed
│   └── 🤖 Agent: Hotel Booking Assistant
│           Status: ✅ Completed
├── 📋 Task: cd8e8d7d-9d5c-44bf-8d30-5a45f4583038
│      Assigned to: Weather Forecast Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Weather Forecast Specialist
│           Status: ✅ Completed
└── 📋 Task: 35381d32-abb8-4dfb-9e65-08ba9822929f
       Assigned to: Travel Itinerary Planner
       Status: ✅ Completed
    └── 🤖 Agent: Travel Itinerary Planner
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 35381d32-abb8-4dfb-9e65-08ba9822929f                                                                     │
│  Agent: Travel Itinerary Planner                                                                                │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 4df25ac7-3665-4a1d-9ee4-14c81d7ec24c                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
from IPython.display import Markdown, display

result = results.tasks_output[-1]
display(Markdown(result.raw))

**Travel Itinerary for Portland, Oregon (April 26 - April 29, 2025)**

---

**Flight Details:**

- **Departure:**
   - **Airline:** Hawaiian Airlines (HA)
   - **From:** Phoenix (PHX)
   - **To:** Portland (PDX)
   - **Date:** April 26, 2025
   - **Time:** 07:00 
   - **Arrival Time:** 09:50 
   - **Duration:** 2 hours 50 minutes
   - **Total Price:** USD 496.58
   - **Available Seats:** 7

- **Return:**
   - **Departure:** Portland (PDX)
   - **Arrival:** Phoenix (PHX)
   - **Date:** April 29, 2025
   - **Time:** 17:25 
   - **Arrival Time:** 19:58 
   - **Duration:** 2 hours 33 minutes

---

**Hotel Accommodation:**

- **Staypineapple, Hotel Rose, Downtown Portland**
   - **Rating:** 4 out of 5 stars
   - **User Rating:** 8.2 Very good (1216 reviews)
   - **Price:** USD 393 (includes taxes and charges)
   - **Availability:** No prepayment needed
   - **Cancellation Policy:** Free cancellation

---

**Weather Forecast for Portland:**

- **April 26, 2025**
   - **Temperature:** High of 9.84°C
   - **Condition:** Overcast clouds
   - **Summary:** Cool start with temperatures around 11.09°C, remaining cloudy and reaching a maximum in the afternoon.

- **April 29, 2025**
   - **Temperature:** Low of 13.87°C
   - **Condition:** Overcast clouds
   - **Summary:** Cloudy sky throughout the day, with nighttime temperatures dropping to about 10.53°C.

---

**Recommended Day-by-Day Itinerary:**

**Day 1: April 26, 2025 (Saturday)**  
- **Morning:** Arrive in Portland at 09:50 AM. Check-in at Staypineapple, Hotel Rose, Downtown Portland.  
- **Afternoon:** Explore downtown Portland, visit the Portland Art Museum and have lunch at a local café.  
- **Evening:** Enjoy dinner at a popular restaurant (suggestion: Le Pigeon). End the day with a stroll along the waterfront.

---

**Day 2: April 27, 2025 (Sunday)**  
- **Morning:** Breakfast at the hotel, followed by a visit to Washington Park (enjoy the Japanese Garden and Rose Test Garden).  
- **Afternoon:** Lunch in the park or nearby food carts. Visit the Oregon Zoo.  
- **Evening:** Dinner at a local farm-to-table restaurant (suggestion: Farm Spirit).

---

**Day 3: April 28, 2025 (Monday)**  
- **Morning:** Breakfast at nearby café (suggestion: Tōv Coffee).  
- **Afternoon:** Take a trip to Columbia River Gorge for breathtaking views and hiking opportunities (Multnomah Falls is a must-see).  
- **Evening:** Return to Portland and enjoy dinner at a brewery (suggestion: Deschutes Brewery).

---

**Day 4: April 29, 2025 (Tuesday)**  
- **Morning:** Have breakfast at the hotel and check-out. Last minute shopping or exploring Portland’s Pearl District.  
- **Afternoon:** Head to the airport for your flight; ensure to arrive by 15:00 for your 17:25 flight. Return home with wonderful memories of Portland!

---

**Total Estimated Cost:** 
- **Flight:** $496.58  
- **Hotel:** $393  
**Total= $889.58 for a delightful trip to Portland, Oregon!**  

Have a fantastic journey!